In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import requests

from sklearn.preprocessing import StandardScaler

def apply_pretreatment(dataframe,pretreatment_attrs=None):
    values = json.dumps(dataframe.values.tolist())
    index = json.dumps(dataframe.index.to_list())
    columns = json.dumps(dataframe.columns.to_list())
    pretreatment_attrs = json.dumps(pretreatment_attrs)
    query = f"""
        query {{
            featuresTargetsPretreatment(values: {values}, index: {index}, columns: {columns}, pretreatment_attrs: {pretreatment_attrs}) {{
                success,
                error,
                pretreated_values,
                index,
                columns,
                pretreatment_info
            }}
        }}
    """

    url = "http://172.31.39.13:8000/graphql"
    headers = {
                'Accept-Encoding': 'gzip, deflate, br',
                'Content-Type': 'application/json',
                'Accept': 'application/json',
                'Connection': 'keep-alive',
                'DNT': '1'
            }

    response = requests.post(url, json={"query": query}).json()

    if not response['data']['featuresTargetsPretreatment']['success']:
        print(response['data']['featuresTargetsPretreatment']['error'])
        print(response['data']['featuresTargetsPretreatment'])
    else:
        pretreated_dataframe = pd.DataFrame(
            data = response['data']['featuresTargetsPretreatment']['pretreated_values'],
            columns=response['data']['featuresTargetsPretreatment']['columns'],
            index=response['data']['featuresTargetsPretreatment']['index']
        )

    return pretreated_dataframe, response

In [34]:
dataframe = pd.read_csv('../data/msft_raw.csv',usecols=['DATE','OPCP','HPCP','LPCP','CPCP','ACPCP','VTCP', 'MPN5P'], index_col='DATE')

#dataframe.index = pd.to_datetime(dataframe.index)
dataframe.head()

,OPCP,HPCP,LPCP,CPCP,ACPCP,VTCP,MPN5P
DATE,,,,,,,
2000-02-29,45.87500,45.875000,44.43750,44.68750,27.880445,58437400,46.765625
2000-03-01,44.81250,47.046875,44.46875,45.40625,28.328880,106889800,47.070312
2000-03-02,45.90625,47.687500,45.56250,46.68750,29.128248,106932600,47.656250
2000-03-03,47.37500,49.437500,46.93750,48.06250,29.986110,101435200,48.000000
2000-03-06,48.00000,48.687500,45.06250,45.31250,28.270390,93609400,48.750000


In [35]:
train_end = '2022-10-28'
train = dataframe.loc[:train_end,:]
test = dataframe.loc[train_end:,:].drop(train_end,axis=0)

#### Pretreat Train

In [42]:
pretreated_train, ftp_response = apply_pretreatment(dataframe=train)
pretreatment_object = json.loads(ftp_response['data']['featuresTargetsPretreatment']['pretreatment_info'])

pretreated_train.head()

TypeError: 'NoneType' object is not subscriptable

In [37]:
pretreatment_object

{'OPCP': {'S': '{"copy": true, "n_features_in_": 1, "n_samples_seen_": 5705, "max_abs_": 344.62, "scale_": 344.62}',
  'T': '{"n_quantiles": 1000, "output_distribution": "uniform", "ignore_implicit_zeros": false, "subsample": 10000, "random_state": null, "copy": true, "n_features_in_": 1, "n_quantiles_": 1000, "references_": [0.0, 0.001001001001001001, 0.002002002002002002, 0.003003003003003003, 0.004004004004004004, 0.005005005005005005, 0.006006006006006006, 0.007007007007007007, 0.008008008008008008, 0.009009009009009009, 0.01001001001001001, 0.011011011011011011, 0.012012012012012012, 0.013013013013013013, 0.014014014014014014, 0.015015015015015015, 0.016016016016016016, 0.017017017017017015, 0.018018018018018018, 0.01901901901901902, 0.02002002002002002, 0.02102102102102102, 0.022022022022022022, 0.023023023023023025, 0.024024024024024024, 0.025025025025025023, 0.026026026026026026, 0.02702702702702703, 0.028028028028028028, 0.029029029029029027, 0.03003003003003003, 0.03103103103

#### Scale Test

In [38]:
pretreated_test, ftp_response = apply_pretreatment(dataframe=test,pretreatment_attrs=json.dumps(pretreatment_object))

pretreated_test.head()

,OPCP,HPCP,LPCP,CPCP,ACPCP,VTCP,MPN5P
2022-10-31,0.926189,0.924557,0.925871,0.924717,0.926618,0.226262,0.918809
2022-11-01,0.926649,0.925213,0.923512,0.922573,0.923743,0.262225,0.918809
2022-11-02,0.922983,0.922223,0.919440,0.918101,0.919968,0.387370,0.920216
2022-11-03,0.918016,0.916794,0.915259,0.911312,0.916388,0.359542,0.921523
2022-11-04,0.916597,0.917328,0.914401,0.918639,0.920761,0.362681,0.922134
